In [1]:
import pandas as pd
import os
import pickle

In [3]:
# path in which train.csv and test.csv are located

minipath = os.getcwd()

if not os.path.exists(os.path.join(minipath, 'data')):
    os.mkdir(os.path.join(minipath, 'data'))
    
    
PATH = os.path.join(minipath, 'data')

## Download data

In [24]:
!wget 'https://meli-data-challenge.s3.amazonaws.com/test.csv' -P $PATH

--2019-10-07 21:20:29--  https://meli-data-challenge.s3.amazonaws.com/test.csv
Resolving meli-data-challenge.s3.amazonaws.com (meli-data-challenge.s3.amazonaws.com)... 52.216.147.19
Connecting to meli-data-challenge.s3.amazonaws.com (meli-data-challenge.s3.amazonaws.com)|52.216.147.19|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16467436 (16M) [text/csv]
Saving to: ‘/home/michel/Documents/MachineLearning/kaggle/meli/github/data/test.csv’

test.csv            100%[===================>]  15.70M  4.38MB/s    in 3.6s    

2019-10-07 21:20:34 (4.38 MB/s) - ‘/home/michel/Documents/MachineLearning/kaggle/meli/github/data/test.csv’ saved [16467436/16467436]



In [25]:
!wget 'https://meli-data-challenge.s3.amazonaws.com/train.csv.gz' -P $PATH

--2019-10-07 21:20:34--  https://meli-data-challenge.s3.amazonaws.com/train.csv.gz
Resolving meli-data-challenge.s3.amazonaws.com (meli-data-challenge.s3.amazonaws.com)... 52.216.147.19
Connecting to meli-data-challenge.s3.amazonaws.com (meli-data-challenge.s3.amazonaws.com)|52.216.147.19|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 691651384 (660M) [application/x-gzip]
Saving to: ‘/home/michel/Documents/MachineLearning/kaggle/meli/github/data/train.csv.gz’

train.csv.gz        100%[===================>] 659.61M  5.88MB/s    in 2m 5s   

2019-10-07 21:22:40 (5.28 MB/s) - ‘/home/michel/Documents/MachineLearning/kaggle/meli/github/data/train.csv.gz’ saved [691651384/691651384]



In [26]:
train_file = os.path.join(PATH, 'train.csv.gz')

!gunzip  $train_file

## Make all the directories

In [ ]:
# make directories if don't exist


if not os.path.exists(os.path.join(PATH, 'models')):
    os.mkdir(os.path.join(PATH, 'models'))
if not os.path.exists(os.path.join(PATH, 'vocabularies')):
    os.mkdir(os.path.join(PATH, 'vocabularies'))
if not os.path.exists(os.path.join(PATH, 'submissions')):
    os.mkdir(os.path.join(PATH, 'submissions'))
if not os.path.exists(os.path.join(PATH, 'datasets')):
    os.mkdir(os.path.join(PATH, 'datasets'))
if not os.path.exists(os.path.join(PATH, 'stats')):
    os.mkdir(os.path.join(PATH, 'stats'))

    
PATH_DATASET = os.path.join(PATH, 'datasets')

PATH_VOCAB = os.path.join(PATH, 'vocabularies')
    

# Load data

In [ ]:
# 
path = '/home/gartner/Documents/meli/github/data'

file = os.path.join(PATH, 'train.csv')
train = pd.read_csv(file)

file = os.path.join(PATH, 'test.csv')
test = pd.read_csv(file)


# Make classes dictionaries


In [ ]:
import pandas as pd
import pickle

#subset = pd.read_hdf('train.h5')
subset = train.drop(columns=['language', 'label_quality'])

cat_dict = {}
for i,category in enumerate(subset.category.unique()):
    cat_dict[category] = i
    
cat_dict_inv = []
for i,category in enumerate(subset.category.unique()):
    cat_dict_inv.append(category)
    
counts = subset.groupby('category').count()

values = counts.values
categories = counts.index

dict_count = {}
for value,category in zip(values,categories):
    dict_count[cat_dict[category]] = value[0]
    
    
classes_dict = {}

classes_dict['class_stoi'] = cat_dict
classes_dict['class_itos'] = cat_dict_inv
classes_dict['class_count'] = dict_count

with open(os.path.join(PATH,'classes_dict.pkl'), 'wb') as f:
    pickle.dump(classes_dict, f)   

## make classes weights

In [ ]:
train_unrel = train[train.label_quality == 'unreliable']
train_rel = train[train.label_quality == 'reliable']


class_rel = list(train_rel.category.unique())
class_unrel = list(train_unrel.category.unique())

list_train_unrel = list(train_unrel.groupby('category'))
list_train_rel = list(train_rel.groupby('category'))

a = train_rel.groupby('category').count().title
b = train_unrel.groupby('category').count().title



In [ ]:
import numpy as np

len(cat_dict_inv), cat_dict[a.index[1000]], cat_dict[b.index[1000]]

weights = np.zeros((len(cat_dict_inv), 3))

    
for i in range(len(a)):
    # Category
    cat = a.index[i]
    # idx in convention
    idx = cat_dict[cat]
    # Count
    count = a[i]
    
    weights[idx,0] = count
    
for i in range(len(b)):
    # Category
    cat = b.index[i]
    # idx in convention
    idx = cat_dict[cat]
    # Count
    count = b[i]
    
    weights[idx,1] = count
    
for i in range(len(cat_dict_inv)):
    weights[i,2] = weights[i,0] / weights[i,1] 

    
with open( os.path.join(PATH, 'classes_weights.pkl'), 'wb') as f:
    pickle.dump(weights, f)     


# Make vocabularies

## make vocabulary using training set

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pickle


# Load submission test set
train = pd.read_csv( os.path.join(PATH,'train.csv') )


vectorizer = CountVectorizer(
    strip_accents='unicode',
    min_df=1
    )


vector_cat = vectorizer.fit_transform(train.title)

    
vocab = vectorizer.vocabulary_
with open( os.path.join(PATH_VOCAB,'vocab_all.pkl'),'wb') as f:
    pickle.dump(vocab, f)  

## make vocabulary using test set

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pickle

# Load submission test set
test = pd.read_csv( os.path.join(PATH,'test.csv') )


vectorizer = CountVectorizer(
    strip_accents='unicode',
    min_df=1
    )


vector_cat = vectorizer.fit_transform(test.title)

    
vocab = vectorizer.vocabulary_
with open(os.path.join(PATH_VOCAB, 'vocab_test.pkl'),'wb') as f:
    pickle.dump(vocab, f)  

## vocabulary with similar spelling in test set

We assign all words with similar spelling to the same integer

In [ ]:
import pickle

# Load vocabulary
file = open(os.path.join(PATH_VOCAB, 'vocab_test.pkl'), 'rb')
vocab = pickle.load(file)

# Make a list with the vocab
list_vocab = []

for key in vocab:
    list_vocab.append(key)

For each word we look for words in the list that are spelled similar. Then we assign all those words to the same integer and remove all of them from the list.

This preprocessing requires few hours in my cpu...

In [ ]:
from difflib import get_close_matches
import time

new_vocab = {}

# Itero por todos los elementos de list_vocab en ppio
n = 0
start = time.time()
for i in range( len(list_vocab)):

    # Busco matches
    matches = get_close_matches(list_vocab[-1], list_vocab, n=20,  cutoff=0.9)
    
    
    if n%100==0:
        print(matches, time.time()-start)
    
    # En todas las palabras 
    for word in matches:
        new_vocab[word] = i
        
        # Remove words that we've already used
        list_vocab.remove(word)
        n+=1
        
    # We stop when we use all words
    if n==len(vocab):
        break

In [ ]:
# Save 

with open(os.path.join(PATH_VOCAB, 'vocab_test_red.pkl'), 'wb') as f:
    pickle.dump(new_vocab, f)   

## make vocabulary in which each class is greater than 0.0015

for each class, we only keep tokens that appears in more than 0.0015 of the total examples from that class. Takes about 20 minutes in my cpu

In [ ]:
def most_common(vectors, vocab_itos, cant, umbral):
    list_words=[]
    b = vectors.sum(axis=0)
    c = b[b>1]
    more_common = c.argsort()[-3:][::-1]
    n = more_common.size
    for value in range(cant):
        valor = more_common[0, n-1-value]
        #print(value, valor, vocab_itos[valor], b[0, valor])
        
        if (b[0, valor] > max(1,umbral)):
            list_words.append(vocab_itos[valor])
    return list_words

    categories = subset.category.unique()

In [ ]:
fraccion=0.0015
    
vectorizer = CountVectorizer(
    analyzer = 'word',
    strip_accents='unicode',
    min_df=2
    )

list_list_words = []

for i,category in enumerate(categories):
    #print('Leo set: {}'.format(category))
    subset_cat = subset[subset.category==category]


    vector_cat = vectorizer.fit_transform(subset_cat.title)
    vocab_stoi_cat = vectorizer.vocabulary_
    vocab_itos_cat =  {v: k for k, v in vocab_stoi_cat.items()}

    umbral = fraccion * subset_cat.shape[0]
    cant = len(vocab_stoi_cat)
    list_words = most_common(vector_cat, vocab_itos_cat, cant, umbral)

    print(i, category, subset_cat.shape, len(vocab_stoi_cat), len(list_words))

    list_list_words.append(list_words)

    list_vocab = set().union(*list_list_words)


vocab = {}
for i, word in enumerate(list_vocab):
    vocab[word] = i

with open(os.path.join(PATH_VOCAB, 'vocab_0.0015.pkl'), 'wb') as f:
        pickle.dump(vocab, f)    

# Make databases

## Load data

In [ ]:
train = pd.read_csv(os.path.join(PATH,'train.csv'))

file = open(os.path.join(PATH_VOCAB, 'vocab_test.pkl'), 'rb')
vocab_test = pickle.load(file)

file = open(os.path.join(PATH_VOCAB, 'vocab_all.pkl'), 'rb')
vocab_all = pickle.load(file)


## Filter total database

We tokenize each example of the training set using 'vocab_all' and 'vocab_test'. 

We remove all the examples in which the number of the tokens of the title generated using vocab_test is less than (some threshold) times the number of tokens of the title generated using vocab_all 

In [ ]:
vectorizer1 = CountVectorizer(analyzer = 'word',
                             strip_accents='unicode',
                             vocabulary=vocab_test
                            )

tokenizer1 = vectorizer1.build_analyzer()

vectorizer2 = CountVectorizer(analyzer = 'word',
                             strip_accents='unicode',
                             vocabulary=vocab_all
                            )

tokenizer2 = vectorizer2.build_analyzer()

In [ ]:
vectors1 = vectorizer1.fit_transform(train.title)
vectors2 = vectorizer2.fit_transform(train.title)

sumas1 = vectors1.sum(axis=1)
sumas2 = vectors2.sum(axis=1)

In [ ]:
import numpy as np

numbers = np.zeros((vectors1.shape[0], 2))
list_coc = []

for i in range(vectors1.shape[0]):
    numbers[i,0] = sumas1[i,0]
    numbers[i,1] = sumas2[i,0]
    if numbers[i,1] !=0:
        list_coc.append(numbers[i,0] / numbers[i,1])
    else:
        list_coc.append(1.2)
    
umbral1 = 0.9
umbral2 = 0.8
umbral3 = 0.7
umbral4 = 0.6
umbral5 = 0.5

prior = [i for i in range(len(list_coc)) if list_coc[i]==1]
prior1 = [i for i in range(len(list_coc)) if umbral1 <= list_coc[i] < 1]
prior2 = [i for i in range(len(list_coc)) if umbral2 <= list_coc[i] < umbral1]
prior3 = [i for i in range(len(list_coc)) if umbral3 <= list_coc[i] < umbral2]
prior4 = [i for i in range(len(list_coc)) if umbral4 <= list_coc[i] < umbral3]
prior5 = [i for i in range(len(list_coc)) if umbral5 <= list_coc[i] < umbral4]
prior6 = [i for i in range(len(list_coc)) if list_coc[i] < umbral5]
prior7 = [i for i in range(len(list_coc)) if list_coc[i] > 1]

In [ ]:
len(prior), len(prior1), len(prior2), len(prior3), len(prior4), len(prior5), len(prior6)

We make 4 subsets from the train set, including examples with coincidence greater than 0.7, 0.8, 0.9 and equal to 1 respectively 

In [ ]:
train_07 = train.iloc[prior+prior1+prior2+prior3 , :]
train_08 = train.iloc[prior+prior1+prior2 , :]
train_09 = train.iloc[prior+prior1, :]
train_10 = train.iloc[prior, :]

list_train = [train, train_07, train_08, train_09, train_10]
names = ['', '_07', '_08', '_09', '_10']

## Split and save data

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle


for train_set, name in zip(list_train, names):

    X = shuffle(train_set)
    
    # Split data
    X_train, X_test = train_test_split(X, test_size=0.025, stratify=X.category, random_state=3)

    # Reset indexes
    X_train = X_train.reset_index()
    X_test = X_test.reset_index()
    
    
    
    #split data in reliable and unreliable
    train_unrel = X_train[X_train.label_quality == 'unreliable']
    train_rel = X_train[X_train.label_quality == 'reliable']


    class_rel = list(train_rel.category.unique())
    class_unrel = list(train_unrel.category.unique())

    X_train = X_train.drop(columns=['label_quality', 'language', 'index'])
    X_test = X_test.drop(columns=['label_quality', 'language', 'index'])
    train_unrel = train_unrel.drop(columns=['label_quality', 'language', 'index'])
    train_rel = train_rel.drop(columns=['label_quality', 'language', 'index'])

    # Save
    X_train.to_hdf(os.path.join(PATH_DATASET,'train_set{}.h5'.format(name)), key='X_train', mode='w')
    X_test.to_hdf(os.path.join(PATH_DATASET,'test_set{}.h5'.format(name)), key='X_test', mode='w')

    train_unrel.to_hdf(os.path.join(PATH_DATASET,'train_set_unrel{}.h5'.format(name)), key='train_unrel', mode='w')
    train_rel.to_hdf(os.path.join(PATH_DATASET,'train_set_rel{}.h5'.format(name)), key='train_rel', mode='w')